In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
path = '/content/drive/My Drive/aiAssignment2/assignment2/q1a/NaiveBayesClassifier'

In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


In [0]:
def load_file(fileName):
    dataset = pd.read_csv(fileName, header=0, sep=",", encoding="unicode_escape")
    return dataset
  
dataPath = path+'/data/TextClassification_Data.csv'
data = load_file(dataPath)
#data.head(5)

In [0]:
# preprocess creates the term frequency matrix for the review data set
def preprocess(data):
    count_vectorizer = CountVectorizer(stop_words= 'english')
    data = count_vectorizer.fit_transform(data)
    
    #tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data)
    #print(data[0].toarray())
    #print(count_vectorizer.vocabulary_[i[0][1]])
    return data, count_vectorizer

In [7]:
'''
print(data.toarray().shape)
for i in range(data.shape[0]):
    print(np.where(data[i].toarray() > 0)[1])
    print(data[i].toarray()[np.where(data[i].toarray() > 0)[1]])
    break;
'''

'\nprint(data.toarray().shape)\nfor i in range(data.shape[0]):\n    print(np.where(data[i].toarray() > 0)[1])\n    print(data[i].toarray()[np.where(data[i].toarray() > 0)[1]])\n    break;\n'

In [0]:
def learn_model(data, target):
    #Your custom implementation of NaiveBayes classifier will go here.
    classes, counts = np.unique(np.array(target), return_counts=True)
    classCount = dict([(i, j) for i,j in zip(classes, counts)])
    classifier = dict()
    
    seenWordsEachClass = dict([(i, set()) for i in classes])
    #Your custom implementation of NaiveBayes classifier will go here.
    for i, clas in zip(data, target): 
        tempdata = i.toarray()[0]
        nonZeros = np.where(tempdata > 0)[0] #basically index of words present in the current sentence
                                              #(index here is with respect to bag of words)
        for j in nonZeros:
            if j not in seenWordsEachClass[clas]:
                seenWordsEachClass[clas].add(j)
            if j not in classifier:
                classifier[j] = dict([(cl, 0) for cl in classes])
            classifier[j][clas] += tempdata[j]  
    seenWords = set()    
    c = 0
    #Laplacian smoothing:

    for word in range(len(data[0].toarray()[0])):
        if word in classifier:
            if word not in seenWords:
                seenWords.add(word)
                for prob in classifier[word]:  
                      classifier[word][prob] += 1
        else:
            c+=1
            classifier[word] = dict([(cl, 1) for cl in classes])

    for j in classifier: 
        for i in classifier[j]:
          classifier[j][i] /= (classCount[i] + 2)
    return classifier

In [0]:
def classify(classifier, testdata):
    predicted_val=[]
        
    #Your code to classify test data using the learned model will go here
    for i in testdata:
        probClassesGivenSentence = dict([(j, 1) for j in classifier[0]])
        arr = i.toarray()[0]
        nonZeros = np.where(arr > 0)[0]
        probs = dict([(j, classifier[j]) for j in nonZeros])
        print(probs)
        break
        for x in probs:
            for y in probs[x]:               
                probClassesGivenSentence[y] *= probs[x][y]
        prediction = max(probClassesGivenSentence, key=lambda k: probClassesGivenSentence[k]) 
        predicted_val.append(prediction)   
   
    return predicted_val

In [0]:
def evaluate(actual_class, predicted_class):
        
    accuracy = -1    
    #Your code to evaluate the model will go here. The code will print overall model's accuracy and precision 
    #and recall for each class label.
    
    
    print("The accuracy score is :",accuracy)

In [62]:
features = ["SUMMARY", "categories", "sub_categories"]

print("Loading data.....")
dataset = load_file(dataPath)
data, target = dataset[features[0]].fillna(" "), dataset[features[1]]

print("preprocessing data.....")
word_vectors, cv = preprocess(data)
#print(cv.get_feature_names()[2415], cv.get_feature_names()[7090])
trainingX,testX,trainingY,testY = train_test_split(word_vectors,target,test_size=0.4,random_state=43)

#print(trainingX.shape,testX.shape,trainingY.shape,testY.shape)
print("Learning model.....")
model = learn_model(trainingX,trainingY)

print("Classifying test data......")      
predictedY = classify(model, testX)
print(predictedY)
classes, counts = np.unique(np.array(predictedY), return_counts=True)
print(classes, counts)
print("Evaluating results.....")
#evaluate(testY,predictedY)

Loading data.....
preprocessing data.....
Learning model.....
{'APPOINTMENTS': 325, 'ASK_A_DOCTOR': 6, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 6, 'PRESCRIPTION': 5}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 3, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 4}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 2}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 2, 'PRESCRIPTION': 1}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 2}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 2, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 3, 'PRESCRIPTION': 1}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 1}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 2, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 1}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK': 1, 'LAB': 1, 'MISCELLANEOUS': 1, 'PRESCRIPTION': 1}
{'APPOINTMENTS': 1, 'ASK_A_DOCTOR': 1, 'JUNK'

In [12]:
features = ["SUMMARY", "categories", "sub_categories"]

print("Loading data.....")
dataset = load_file(dataPath)
data, target = dataset[features[0]].fillna(" "), dataset[features[1]]
#print(classProbs)

print("preprocessing data.....")
word_vectors = preprocess(data)
trainInd, testInd = train_test_split(range(dataset.shape[0]), test_size=0.4, random_state=43)
print(len(trainInd), len(testInd), dataset.shape)

#print(trainingX.shape,testX.shape,trainingY.shape,testY.shape)
print("Learning model.....")
model = learn_model(trainInd, dataset)

print("Classifying test data......")      
#predictedY = classify(model, testX)

print("Evaluating results.....")
#evaluate(testY,predictedY)

Loading data.....
preprocessing data.....
34368 22912 (57280, 3)
Learning model.....


TypeError: ignored